In [2]:
import numpy as np
import pandas as pd
from modules import *

import os #Used when reading/writing csv files programatically

In [3]:
fec_folder_path = '../data/FEC/'

fec_files = [file for file in os.listdir(fec_folder_path) if os.path.isfile(os.path.join(fec_folder_path, file))]

In [4]:
# Empty list to hold FEC files
FEC_files = []

# format FEC data
print('Formatting FEC data')
for i in fec_files: # Call item in the file list
    
    # Read each file from the FEC file list
    file = pd.read_csv(fr'{fec_folder_path}{i}')
    
    
    # Generate a name for each dataframe based on the filename without the file extension
    name = i.split('.')[0]
    
    # Add the dataframe to the list
    FEC_files.append(file)
print("FEC files formatted")

Formatting FEC data
FEC files formatted


In [ ]:
IL_data = []

for i in IL_files:
    file = pd.read_excel(f'{elec_folder_path}/{i}', sheet_name='TotalsByCounty')
    # file = pd.read_excel(f'{elec_folder_path}/{i}', sheet_name='TotalsByCounty',index_col=0)
    # Generate a name for each dataframe based on the filename without the file extension
    name = f'{i[:7]}' 
    name = name[:7]    
    
    
    # Append both to the empty list
    IL_data.append((name, file))


In [ ]:
#create empty list to hold IL dataframes

IL_cong = []

for year, i in IL_data:
    file = i[i['OfficeName'].str.contains('congress', case=False, na=False)][['County','OfficeName','CanFirstName','CanLastName','Votes']]
    IL_cong.append((year,file))

In [5]:
elec_folder_path = '../data/raw_elec_totals'

IL_files = [file for file in os.listdir(elec_folder_path) if os.path.isfile(os.path.join(elec_folder_path, file)) and file.startswith('IL')]

In [8]:
#create empty list to hold IL dataframes

IL_cong = []

for i in IL_files: # Call item in the file list
    
    # generate name for files
    year = f'{i[:7]}' 
    
    #Read in files
    file = pd.read_excel(f'{elec_folder_path}/{i}', sheet_name='TotalsByCounty')
    
    # Get only data for congressional races, with relevant columns
    file = file[file['OfficeName'].str.contains('congress', case=False, na=False)][['County','OfficeName','CanFirstName','CanLastName','Votes']]
    
    IL_cong.append((year,file))

C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [9]:
# Create empty lists to hold dataframes and their names
formatted_IL = []
IL_names = []

In [10]:
for year, i in IL_cong:   
    # Remove formatting from County names to ensure any differences will not interfere with merging
    i['County'] = i['County'].astype(str).str.lower()
    
    # Format names using 'trim party' function to remove middle names and titles, then concatenate so name matches FEC data
    i['CanFirstName'] = trim_party(i['CanFirstName'], ' ')  # removes middle names
    i['CanLastName'] = trim_party(i['CanLastName'], ' ')  # removes titles appended to last names
    i['Candidate Name(f)'] = i['CanFirstName'] + ' ' + i['CanLastName']
    # format names as lower-case
    i['Candidate Name(f)'] = i['Candidate Name(f)'].astype(str).str.lower()

    # Drop rows where candidates received zero votes
    i = i[i['Votes']!=0]
    
    # Create list of candidates
    cand_list = i['Candidate Name(f)'].unique()

    # Check for multiple offices for each candidate
    for candidate in i['Candidate Name(f)'].unique():
        offices_count = i[i['Candidate Name(f)'] == candidate]['OfficeName'].nunique()
        if offices_count > 1:
            print(f"{candidate} has multiple offices")

    # Since all races appear in a single sheet
    # Create tables for each candidate
    cand_tables = {}
    for cand in cand_list:
        candidate_df = i[i['Candidate Name(f)'] == cand][['County', 'Votes']]
        
        # Check for duplicated counties within each candidate_df
        if candidate_df['County'].duplicated().any():
            raise ValueError(f"Duplicate counties found for candidate {i}")
        
        cand_tables[cand] = candidate_df

    # Merge tables
    merged_df = cand_tables[cand_list[0]]  # the first dataframe is used as the base, all following dataframes will be added
    for cand in cand_list[1:]:
        merged_df = pd.merge(merged_df, cand_tables[cand], on='County', how='outer', suffixes=('_' + cand, ''))  # a full outer join is done so that every county will be added

    # Rename columns to match the candidates
    # The order is preserved by the sequence they had been merged in
    merged_df.columns = ['County'] + list(cand_list)

    # We still need to get rid of any name-final punctuation remaining after removing name suffixes, such as jr. and sr.
    merged_cols = [''.join(char for char in i if char.isalpha() or char.isspace()) for i in list(merged_df.columns)]
    merged_df.columns = merged_cols
    

    # If candidate received zero votes, fill NaN
    merged_df = merged_df.fillna(0)
    formatted_IL.append(merged_df)
    IL_names.append(year)


In [6]:
for i in IL_files: # Call item in the file list
    file = pd.read_excel(f'{elec_folder_path}/{i}', sheet_name='TotalsByCounty')
    
    # Get only data for congressional races
    file = file[file['OfficeName'].str.contains('congress', case=False, na=False)]
    
    
    # Apply IL formatting function
    formatted = format_il(file)

    
    
    # Generate a name for each dataframe based on the filename without the file extension
    name = f'{i}' 
    name = name[:-5]+'_f' 
    
    # Assign the dataframe to the variable name
    globals()[name] = formatted # from the documentation: 'the globals() function is a built-in function that returns a dictionary representing the current global symbol table' only half understand this, but it works (#programming)
    
    # Append both to the empty list creating a list of names and corresponding dataframes
    formatted_IL.append(formatted)
    IL_names.append(name)


C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
C:\Users\joshu\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
formatted_IL[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 54 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   County             102 non-null    object 
 1   donald peloquin    102 non-null    float64
 2   bobby rush         102 non-null    float64
 3   john hawkins       102 non-null    float64
 4   brian woodworth    102 non-null    float64
 5   jesse jackson      102 non-null    float64
 6   marcus lewis       102 non-null    float64
 7   anthony williams   102 non-null    float64
 8   richard grabowski  102 non-null    float64
 9   daniel lipinski    102 non-null    float64
 10  laura anderson     102 non-null    float64
 11  luis gutierrez     102 non-null    float64
 12  hector concepcion  102 non-null    float64
 13  ymelda viramontes  102 non-null    float64
 14  dan schmitt        102 non-null    float64
 15  mike quigley       102 non-null    float64
 16  nancy wade         102 non

In [12]:
FEC_files[0]

,STATE ABBREVIATION,D,CANDIDATE NAME (First),CANDIDATE NAME (Last),CANDIDATE NAME(f),CANDIDATE NAME,PARTY,(I),GENERAL VOTES
0,AL,01,Jo,Bonner,jo bonner,"Bonner, Jo",R,1,196374
1,AL,02,Martha,Roby,martha roby,"Roby, Martha",R,1,180591
2,AL,02,Therese,Ford,therese ford,"Ford, Therese",D,0,103092
3,AL,03,Mike,Rogers,mike rogers,"Rogers, Mike",R,1,175306
4,AL,03,John Andrew,Harris,john harris,"Harris, John Andrew",D,0,98141
...,...,...,...,...,...,...,...,...,...
1532,WY,00,Daniel Clyde,Cummings,daniel cummings,"Cummings, Daniel Clyde",OTHER,0,4963
1533,WY,00,Don,Wills,don wills,"Wills, Don",OTHER,0,3775
1534,WY,S,John,Barrasso,john barrasso,"Barrasso, John",R,1,185250
1535,WY,S,Tim,Chesnut,tim chesnut,"Chesnut, Tim",D,0,53019


In [13]:
# Create zipped list of formatted State Election data, FEC list of candidates and parties, and the filenames found in raw_elec data
zipped_IL_FEC = zip(formatted_IL, FEC_files, IL_names)

In [14]:
# Further process and transform election data, grouping vote totals by party and incumbancy
# Allows analysis on these two metrics
for i, j, k in zipped_IL_FEC:
    
    # Joins FEC and State data for each year, produces list of counties as well
    # If an error is generated here, there is likely a mismatch between the counties in these files
    formatted_IL_FEC, counties = state_join_FEC(i,j)
    transformed_data = state_trans(formatted_IL_FEC, counties)
    
    # Writes the transformed data to a .csv file whose name references the original filename
    transformed_data.to_csv(fr"../data/formatted_house_totals/{k[:7]}.csv", index=False)
    



In [24]:
IL_formatted_path = '../data/formatted_house_totals/'

IL_files_f = [file for file in os.listdir(IL_formatted_path) if os.path.isfile(os.path.join(IL_formatted_path, file)) and file.startswith('IL')]

formatted_IL_fec = []

for i in IL_files_f: # Call item in the file list
    
    # Read each file from the FEC file list
    file = pd.read_csv(fr'{IL_formatted_path}/{i}')
    
    
    # Generate a name for each dataframe based on the filename without the file extension
    name = i.split('.')[0]
    
    # Add the dataframe to a dictionary with the key being the filename
    formatted_IL_fec.append((name,file))

In [25]:
formatted_IL_fec

[('IL_2012',
         County        D0         D1    OTHER0        R0        R1
  0        cook  111158.0  1278631.0  113405.0  335037.0  106321.0
  1        will   90973.0    39097.0    3075.0   59899.0   72222.0
  2    kankakee       0.0    11828.0    5165.0   27370.0       0.0
  3      dupage  168250.0    12602.0    1141.0   16130.0  191617.0
  4        kane   81574.0        0.0       0.0       0.0   94320.0
  ..        ...       ...        ...       ...       ...       ...
  97     morgan    3566.0        0.0       0.0       0.0   10257.0
  98       pike    1656.0        0.0       0.0       0.0    5522.0
  99   schuyler    1069.0        0.0       0.0       0.0    2771.0
  100     scott     617.0        0.0       0.0       0.0    1914.0
  101  woodford    3602.0        0.0       0.0       0.0   15036.0
  
  [102 rows x 6 columns]),
 ('IL_2014',
         County       D0        D1   OTHER0        R0       R1
  0        lake  17615.0   69821.0      0.0   74436.0  35379.0
  1        coo